In [121]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os, string
import csv
import pymongo
import time

# Extract (Scrape) Data

#### *Does not need to be loaded to pymongo* - can be scraped on the spot for the Project 2

In [2]:
# currentDirectory = os.path.dirname(__file__)
# executable_path = {"executable_path": f"{currentDirectory}\chromedriver.exe"}
executable_path = {'executable_path': 'chromedriver.exe'} #notebook version
browser = Browser('chrome', **executable_path, headless=True)

### Scrape Schedules

In [59]:
nba_sched_url = "https://www.espn.com/nba/schedule/_/date/"
browser.visit(nba_sched_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')
sched_tables = soup.find('table', class_="schedule has-team-logos align-left")
schedules_sites = pd.read_html(sched_tables.prettify())
schedules_sites

[            matchup          Unnamed: 1 time  (PT) nat tv  \
 0     Portland  POR        Indiana  IND       LIVE    TNT   
 1   Sacramento  SAC  Oklahoma City  OKC       LIVE    NaN   
 2  Los Angeles  LAL    Golden State  GS    7:30 PM    TNT   
 
                  tickets  Unnamed: 5  
 0     Buy on Vivid Seats         NaN  
 1     Buy on Vivid Seats         NaN  
 2  Tickets as low as $36         NaN  ]

# Load - Transform

### Load dump to mongo / Clean on the go

In [111]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.nba
# db.nba_players_stg.drop()

In [ ]:
base_reference_url = "https://www.landofbasketball.com"
#Traverse to all players
for letter in string.ascii_lowercase[:26]:
    players_url = f"https://www.landofbasketball.com/nba_players_index/letter_{letter}.htm"
    browser.visit(players_url)
    browser.html
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    all_player_table = soup.find_all('div', class_="indice-items")
    for row in all_player_table:
        player_d = {}
        print(row.find('a')['href'], end='')
        player_url = row.find('a')['href'].replace('..', base_reference_url)
        tables = pd.read_html(player_url)
        try_df = tables[1].set_index(0)
        try_df.rename(columns={0: "Key", 1: "Value"}, inplace=True)
        try_df = try_df.to_dict('index')
        player_d['Name'] = tables[3][0][0].split(' Profile',1)[0]
        player_d['Data'] = try_df
        db.nba_players_stg.insert_one(player_d)

### Validate data availability

In [116]:

# result_dict['facts'] = list(db.mars_fact.find({}, {"_id": 0}))
for a in db.nba_players_stg.find({}, {"_id": 0}).limit(2):
    print(a)

{'Name': 'Alaa Abdelnaby', 'Data': {'Position:': {'Value': 'Forward / Center'}, 'Jersey Number:': {'Value': '31 (Trail Blazers), 5 (Bucks), 4 (Celtics), 30 (Kings, 76ers)'}, 'Height:': {'Value': '6-10 / 2.08 mts'}, 'Weight:': {'Value': '240 lbs / 108.9 kg'}, 'Born:': {'Value': 'June 24, 1968 in Cairo, Egypt'}, 'High-School:': {'Value': 'Bloomfield in Bloomfield, New Jersey'}, 'College:': {'Value': 'Duke University'}, 'Drafted:': {'Value': 'by the Portland Trail Blazers with No. 25 overall pick in round 1 of the 1990 NBA Draft'}, 'NBA Seasons:': {'Value': '5 (3 Playoffs)'}, 'NBA Titles:': {'Value': 'none'}, 'All-Star:': {'Value': 'none'}}}
{'Name': 'Zaid Abdul-Aziz', 'Data': {'Position:': {'Value': 'Center / Forward'}, 'Jersey Number:': {'Value': '21 (Royals), 16 (Bucks), 35 (Bucks, Supersonics), 6 (Rockets), 35 (Rockets, Braves), 54 (Celtics), 27 (Rockets)'}, 'Height:': {'Value': '6-9 / 2.06 mts'}, 'Weight:': {'Value': '235 lbs / 106.6 kg'}, 'Born:': {'Value': 'April 07, 1946 in Brookl

In [127]:
nba_news_url = "https://apnews.com/NBA"
browser.visit(nba_news_url)
browser.html
# time.sleep(10)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_tables = soup.find_all('div', attrs={"data-key":"feed-card-wire-story-with-image"})
# schedules_sites = pd.read_html(sched_tables.prettify())
# schedules_sites


In [131]:
for row in news_tables:
    print(row.h1.text) # Headlines
    print(row.p.text)

Gallinari scores 24; Thunder rally from 19 down to top Kings
OKLAHOMA CITY (AP) — Danilo Gallinari scored 24 points and the Oklahoma City Thunder rallied from 19 points down in the third quarter to beat the Sacramento Kings 112-108 on Thursday...
Sabonis scores 20 points, Pacers beat Trail Blazers, 106-100
INDIANAPOLIS (AP) — Domantas Sabonis had 20 points and 11 rebounds to lead the Indiana Pacers to a 106-100 victory over the Portland Trail Blazers on Thursday night.
Minus Embiid and Simmons, Harris leads 76ers past Knicks
PHILADELPHIA (AP) — Tobias Harris’ performance had the 76ers feeling good during the game, and the doctor’s report on Joel Embiid afterward had them feeling even better.
LaRose carries Sacred Heart past St. Francis Brooklyn 73-63
FAIRFIELD, Conn. (AP) — Kinnon LaRose had a season-high 21 points as Sacred Heart topped St. Francis Brooklyn 73-63 on Thursday night.
76ers center Joel Embiid sidelined by shoulder injury
PHILADELPHIA (AP) — The Philadelphia 76ers are dow